# 动作流（FlowAction）设计和实现中包含的函数式编程思想（草稿）

## -1. 关于主题

- **为什么要这么写？**
- 函数式编程

## 0. 关于动作流
- 前置动作，实现逻辑和后置动作，称为流(Flow)。
- 调用服务(InvokeService), 打开视图(OpenView), 关闭页面(Close), 跳转（Goto）称为动作（Action）
两者合起来称为 (FlowAction)

## 1. 动作流的Schema描述

### 前置节点 - Validate

In [5]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

### 逻辑节点 - Compute

In [6]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProducts',
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

### 后置节点 - Effect

In [7]:
const afterFlow = {
    type: 'Flow',
    name: 'Effect',
    props: {},
    children: [
        {
            type: 'Action',
            name: 'OpenView',
            props: {
                viewKey: 'sayHello'
            }
        }
    ]
}

### Pipeline

In [8]:
const pipeline = {
    type: 'Flow',
    name: 'Pipeline',
    children: [
        beforeFlow,
        computeFlow,
        afterFlow
    ]
}

In [9]:
pipeline

{
  type: 'Flow',
  name: 'Pipeline',
  children: [
    {
      type: 'Flow',
      name: 'Validate',
      props: [Object],
      children: [Array]
    },
    {
      type: 'Flow',
      name: 'Compute',
      props: [Object],
      children: [Array]
    },
    { type: 'Flow', name: 'Effect', props: {}, children: [Array] }
  ]
}


### 类型定义

In [10]:
type ActionNode = {
    type: 'Action',
    name: string,
    props: Record<string, unknown>
}

type FlowNode = {
    type: 'Flow',
    name: string,
    props: Record<string, unknown>
    children: (FlowNode | ActionNode)[]  // 这里支持嵌套
}

### 动作的定义

- 什么是动作？如何定义动作？
- 动作是一个操作，操作属于一种计算，函数即计算，所以动作可以用函数表达。—— 动作即函数！

### trival version

In [21]:
const validateData = (ctx, props) => {
    const record = ctx.record
    // 实际上会解析规则来执行
    return props.validate(record)
}



In [24]:
const invokeService = async (ctx, props) => {
    const route = ctx.route
    return props.invokeService(props.serviceKey)
}

In [28]:
const ctx = {
    record: { id: '1', name: 'iphone' },
    route: { module: 'products', action: 'show' }
}

### 问题和想法

- 结构 vs 流程 （数据结构即算法）
- 组合

## 函数组合 Function Composition

- 什么样的函数可组合？

### 函数的类型

- 什么是函数的类型？

#### 数据类型

In [17]:
[1, true, 'string']

[ 1, true, 'string' ]


In [20]:
const x2 = x => x * 2
const add = (x, y) => x + y

In [35]:
// (x + 1) * 2

const f = x => x2(add1(x))
f(100)

202


In [39]:
const compose = (f, g) => x => f(g(x)) 

In [36]:
const f = compose(x2, add1)

In [37]:
f(100)

202


### curry

In [48]:
const add = (x, y, z) => x + y + z
const add2 = x => y => z => x + y + z

In [49]:
add(1,2,3)

6


In [50]:
add2(1)(2)(3)

6


### pointless

In [30]:
const range = n => [...Array(n).keys()]

In [52]:
range(10)

[
  0, 1, 2, 3, 4,
  5, 6, 7, 8, 9
]


In [27]:
const { faker } = require('@faker-js/faker')

[faker-js](https://fakerjs.dev/guide/usage.html) [faker-js/api](https://fakerjs.dev/api/)

In [28]:
faker.animal.cat()

Thai


In [31]:
const cats = range(100).map(_ => ({ id: faker.datatype.uuid(), name: faker.animal.cat(), age: faker.datatype.number(100), dark: faker.datatype.boolean() }))

In [32]:
cats.filter(x => x.age < 10).map(x => x.id)

[
  'ef0beef6-2be0-4201-a2a2-f5633e093a31',
  '6e76ba67-e998-4d89-b834-b73b858f7af1',
  '6a4edfbd-3b59-4771-a558-09db92faddf0',
  '0c3f85e7-20dc-407c-9308-d670aa8b310e',
  '64f7c474-647c-49a1-a761-862c58f7cb1d',
  '309ca852-4ee9-44c9-8707-9006cc1ac1bc',
  'a44dccf5-2864-4cc4-967e-e5f376a49e06',
  '388efb5c-4d0e-4810-b6aa-46df507835ea',
  'f1fb1ab9-12d9-4d65-8fe8-3d255d9126b1'
]


In [33]:
const findByAge = (list, age) => list.filter(x => x.age < age)

In [94]:
//findByAge(cats, 10)

In [34]:
const findByAge = list => age => list.filter(x => x.age < age)

//fiterWithAge(cats)(10)
findByAge(cats)

[Function (anonymous)]


In [35]:
const search = list => text => list.filter(x => x.name.includes(text))

In [98]:
//search(cats)('S')
search(cats)

[Function (anonymous)]


In [36]:
const fiindByAgeAndSearch = (list, age, text) => search(findByAge(list)(age))(text)

In [115]:
// fiindByAgeAndSearch(cats, 10, 'S')

In [37]:
const findByAge = age => list => list.filter(x => x.age < age)
const search = text => list => list.filter(x => x.name.includes(text))

In [40]:
const fiindByAgeAndSearch = (age, text) => compose(search(text), findByAge(age))

In [41]:
fiindByAgeAndSearch(10, 'S')(cats)

[
  {
    id: '64f7c474-647c-49a1-a761-862c58f7cb1d',
    name: 'British Shorthair',
    age: 9,
    dark: true
  },
  {
    id: '309ca852-4ee9-44c9-8707-9006cc1ac1bc',
    name: 'Snowshoe',
    age: 7,
    dark: true
  },
  {
    id: '388efb5c-4d0e-4810-b6aa-46df507835ea',
    name: 'Siamese',
    age: 2,
    dark: false
  }
]


### datalast

In [123]:
const filter = f => list => list.filter(f)
const map = f => list => list.map(f)
const take = n => list => list.slice(0, n)

In [127]:
const compose = (a, b, c) => x => a(b(c(x))) 

In [128]:
const get2DarkCatsWithS = compose(
    take(4),
    filter(x => x.name.includes('S')),
    filter(x => x.dark),
)

In [129]:
get2DarkCatsWithS(cats)

[
  {
    id: '8e53baad-c931-4e2b-a44e-2313e99c6654',
    name: 'Sphynx',
    age: 2,
    dark: true
  },
  {
    id: 'e638e320-2046-4e1d-92ce-86b6b484b830',
    name: 'Savannah',
    age: 85,
    dark: true
  },
  {
    id: '516121ab-c3fa-4700-b00d-8a89ee41d860',
    name: 'British Shorthair',
    age: 29,
    dark: true
  },
  {
    id: 'aadbc7da-83e7-4661-9ef6-f9888e635b26',
    name: 'British Shorthair',
    age: 43,
    dark: true
  }
]


## use ramda

In [142]:
const { map, filter, compose, prop, take, pick, sortBy } = require('ramda')

### transform data

In [140]:
const transform = map(pick(['id', 'name']))
// transform(cats)

In [160]:
const getData = compose(
    take(5),
    map(pick(['id', 'name'])),
    sortBy(prop('age')),
    filter(x => x.age > 10)
)

getData(cats)

[
  { id: '405c6691-fd40-45c3-a312-5af5c94f3939', name: 'Thai' },
  { id: '05236427-91df-41a1-addc-1839d76cb6e2', name: 'Burmese' },
  { id: 'c727e101-a862-4250-b3ed-bb6ca8d7eae1', name: 'Burmese' },
  { id: '112fe575-4c18-46c5-9e0e-35efd5f0efc0', name: 'Himalayan' },
  { id: '5fbc6332-1271-4a63-890c-67a29077dd31', name: 'Singapura' }
]


### curry

In [163]:
const add = a => b => c => a + b + c
add(1)(2)(3)

6


In [166]:
const { curry } = require('ramda')

In [168]:
const add = curry((a, b, c) => a + b + c)
add(1, 2, 3)
add(1, 2)(3)
add(1)(2, 3)
add(1)(2)(3)

6


## 组合动作函数

### schema node

In [169]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

In [170]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProducts',
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

### 动作的函数类型

#### 可组合: curry, datalast, arity 1

In [171]:
const invokeService = props => ctx => {
    
}

const invokeFlow = props => ctx => {
}

// or
const validateData = (a, b, c) => ctx => {
}

#### async

In [229]:
const sleep = n => new Promise(r => setTimeout(r, n))

const invokeService = props => async ctx => {
    await sleep(300)
    console.log('invoke service, props: %o, ctx: %o', props, ctx)
    return { serviceKey: props.serviceKey, param: props.param, route: ctx.route }
}

In [230]:
const invokeFlow = props => async ctx => {
    await sleep(300)
    console.log('invoke flow, props: %o, ctx: %o', props, ctx)
    return { flowKey: props.flowKey, route: ctx.route }
}

In [231]:
const ctx = {
    record: { id: '1', name: 'iphone' },
    route: { module: 'products', action: 'show' }
}

#### test

The code should be written that easy to test, but ideally it should not require testing.

In [232]:
await invokeService({ serviceKey: 'xxxbbb', param: '123' })(ctx)

invoke service, props: { serviceKey: 'xxxbbb', param: '123' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
{
  serviceKey: 'xxxbbb',
  param: '123',
  route: { module: 'products', action: 'show' }
}


## 实现Flow

- 动作的组合

In [233]:
const computeFlow = {
    type: 'Flow',
    name: 'Compute',
    props: {
        parallel: true,
    },
    children: [
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProduct',
                param: 1,
            }
        },
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'loadProduct',
                param: 2,
            }
        },
        {
            // 工作流
            type: 'Action',
            name: 'InvokeFlow',
            props: {
                flowKey: 'notifyDingding'
            }
        }
    ]
}

函数即数据， 将上述结构转成函数

In [234]:
const Actions = {
  InvokeService: invokeService,
  InvokeFlow: invokeFlow
}

In [219]:
Actions

{
  InvokeService: [Function: invokeService],
  InvokeFlow: [Function: invokeFlow]
}


In [235]:
const fns = computeFlow.children.map(child => Actions[child.name](child.props))

In [221]:
fns

[
  [AsyncFunction (anonymous)],
  [AsyncFunction (anonymous)],
  [AsyncFunction (anonymous)]
]


In [222]:
type Context = unknown
type ActionFn = (ctx: Context) => unknown

### Flow的函数类型

In [223]:
type Dict = Record<string, unknown>
const compute = (props: Dict, children: ActionFn[]) => async (ctx: Context) => {
  
}

### 并行 & map

In [ ]:
const compute = (props, children) => async ctx => {
    return Promise.all(children.map(fn => fn(ctx))) 
}

In [ ]:
await compute({}, fns)(ctx)

invoke service, props: { serviceKey: 'loadProduct', param: 1 }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
invoke service, props: { serviceKey: 'loadProduct', param: 2 }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
invoke flow, props: { flowKey: 'notifyDingding' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
[
  {
    serviceKey: 'loadProduct',
    param: 1,
    route: { module: 'products', action: 'show' }
  },
  {
    serviceKey: 'loadProduct',
    param: 2,
    route: { module: 'products', action: 'show' }
  },
  {
    flowKey: 'notifyDingding',
    route: { module: 'products', action: 'show' }
  }
]


### 串行 & reduce

In [ ]:
const compute = (props, children) => async ctx => {
    return children.reduce((promise, fn) => promise.then(x => fn({ ...ctx, ...x })), Promise.resolve({}))
}

In [ ]:
await compute({}, fns)(ctx)

invoke service, props: { serviceKey: 'loadProduct', param: 1 }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}
invoke service, props: { serviceKey: 'loadProduct', param: 2 }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' },
  serviceKey: 'loadProduct',
  param: 1
}
invoke flow, props: { flowKey: 'notifyDingding' }, ctx: {
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' },
  serviceKey: 'loadProduct',
  param: 2
}
{
  flowKey: 'notifyDingding',
  route: { module: 'products', action: 'show' }
}


## 加深对reduce的理解

In [240]:
const list = range(100).map(_ => Math.random())

In [242]:
// list

In [243]:
list.reduce((acc, item) => acc + item, 0)

48.07869205749073


In [257]:
// cats

In [260]:
// 组织成以age为key的Map列表

cats.reduce((acc, cat) => {
  const bag = acc[cat.age] || []
  acc[cat.age] = bag
  bag.push(cat)
  return acc
}, {})

''


In [268]:
const { groupBy, prop } = require('ramda')

groupBy(prop('age'))(cats); //(cats)

''

In [276]:
const { reduce } = require('ramda')

const add = (a, b) => a + b
reduce(add, 0, list)

48.07869205749073


In [279]:
const sumCatsAge = compose(
  reduce(add, 0),
  map(prop('age'))
)

sumCatsAge(cats)


5148


### break in reduce

In [ ]:
const { reduced, reduce } = require('ramda')

In [308]:
const reducer = (a, b) => a > 1000 ? reduced(a) : a + b
reduce(reducer, 0)      (range(100))

// 如何实现呢？

1035


### compose

In [289]:
const compose2 = (f, g) => x => f(g(x))
const compose = (...fns) => fns.reduce(compose2, x => x)

In [286]:
const add = a => b => a + b

compose(
  add(1),
  add(2),
  add(3),
  add(4)
)(10)

20


In [292]:
const compose = (...args) => reduce(compose2, x => x, args)

In [294]:
compose(
  compose(
    add(1),
    add(2),
  ),
  compose(
    compose(
      add(3),
      add(4),
    ),
    add(5)
  ),
  add(6)
)(0)

21


## 实现 Validation Flow

In [300]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
              // 规则选择器的属性值
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
            }
        }
    ]
}

### Validate Flow的实现

In [335]:
const validateFlow = (props, actions) => async ctx => {
    for (const fn of actions) {
        const ret = await fn(ctx)
        if (!ret) {
          return false
        }
    }
    return true
}

In [336]:
ctx

{
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}


In [342]:
const beforeFlow = {
    type: 'Flow',
    name: 'Validate',
    props: {
        // 这里只是演示，实际中不需要，直接实现成 breakOnFirst即可满足需求
        breakOnFirst: true,
    },
    children: [
        {
            type: 'Action',
            name: 'ValidateData',
            props: {
                name: 'iphone'
            }
        },
        
        {
            type: 'Action',
            name: 'InvokeService',
            props: {
                serviceKey: 'xxx',
                id: '1'
            }
        }
    ]
}

In [343]:
const ValidateData = props => async ctx => ctx.record.name === props.name
const InvokeService = props => async ctx => ctx.record.id === props.id

const Actions = {
  ValidateData,
  InvokeService,
}

In [344]:
const fns = beforeFlow.children.map(child => Actions[child.name](child.props))

fns

[ [AsyncFunction (anonymous)], [AsyncFunction (anonymous)] ]


In [345]:
ctx

{
  record: { id: '1', name: 'iphone' },
  route: { module: 'products', action: 'show' }
}


In [346]:
await validateFlow(beforeFlow.props, fns)  (ctx)

true


## compile - 组合起来

### Schema

In [351]:
type FlowActionNode = ActionNode | FlowNode

type Dict = Record<string, unknown>

type ActionNode = {
    type: 'Action',
    name: string,
    props: Dict,
}

type FlowNode = {
    type: 'Flow',
    name: string,
    props: Dict,
    children: FlowActionNode[]
}

### API

#### 之前的定义

In [357]:
type Transformer = (ctx: Context) => Promise<unknown>

type ActionFn = (props: Dict) => Transformer
type FlowFn = (props: Dict, transformers: Transformer[]) => Transformer

#### 使用方法

In [366]:

// 定义Action函数集 
const Actions: Record<string, ActionFn> = { }

// 定义流函数集
const Flows: Record<string, FlowFn> = {}

// 构建编译器
const compiler = createCompiler({ Actions, Flows })  // 只有这个没实现

// 创建转换器
const transformer = compiler(schemaNode)

// 使用
const ctx = {}
const result = await transformer(ctx)

12:21 - This expression is not callable.
12:21 - Type 'void' has no call signatures.
12:30 - Cannot find name 'schemaNode'.


### compiler的实现

- tree的应用
- 简单递归

### 思考类型

In [409]:
type Library = {
    Actions: Record<string, ActionFn>
    Flow: Record<string, FlowFn>
}

type CompilerFactory = (library: Library) => Compiler
type Compiler = (node: FlowActionNode) => Transformer

### 最简可运行实现

In [444]:
const createCompiler = ({ Actions, Flows }) => {
    const compiler = node => {
        console.log('compile', node)
        if (node.type !== 'Flow') {
            return Actions[node.name](node.props)
        }
        const transformers = node.children.map(compiler)
        return Flows[node.name](node.props, transformers)
    }
    return compiler
}

#### 定义FlowAction

In [445]:
const sleep = n => new Promise(r => setTimeout(r, n))

// 调用服务
const InvokeService = props => async ctx => {
    await sleep(300)
    console.log('invoke service, props: %o, ctx: %o', props, ctx)
    return { serviceKey: props.serviceKey, param: props.param, route: ctx.route }
}

const InvokeFlow = props => async ctx => {
    await sleep(300)
    console.log('invoke flow, props: %o, ctx: %o', props, ctx)
    return { flowKey: props.flowKey, route: ctx.route }
}

// 验证数据
const ValidateData = props => async ctx => ctx.record.name === props.name

// 验证服务
const ValidateService = props => async ctx => ctx.record.id === props.id

const Actions = {
    InvokeService,
    InvokeFlow,
    ValidateData,
    ValidateService,
}

const ValidateFlow = (props, transformers) => async ctx => {
    for (const fn of transformers) {
        const ret = await fn(ctx)
        if (!ret) {
          return false
        }
    }
    return true
}

const ComputeFlow = (props, transformers) => async ctx => {
    return transformers.reduce((promise, fn) => promise.then(x => fn({ ...ctx, ...x })), Promise.resolve({}))
}

const EffectFlow = (props, transformers) => async ctx => {
    await Promise.all(transformers.map(tranform => transform(ctx)))
    return null
}

const Flows = {
    Validate: ValidateFlow,
    Compute: ComputeFlow,
    Effect: EffectFlow,
    Pipeline: ValidateFlow,
}

### 使用

In [448]:
const compiler = createCompiler({ Actions: Actions, Flows: Flows })

In [449]:
const node = {
  type: 'Flow',
  name: 'Pipeline',
  children: [
      {
          type: 'Flow',
          name: 'Validate',
          children: [
              {
                  name: 'ValidateData',
                  props: { name: 'iphone' }
              },
              {
                  name: 'ValidateService',
                  props: { id: '123' }
              }
          ]
      },
      {
          type: 'Flow',
          name: 'Compute',
          children: [
              {
                  name: 'InvokeService',
                  props: { serviceKey: 'loadProduct', param: 1 }
              },
              
              {
                  name: 'InvokeService',
                  props: { serviceKey: 'loadProduct', param: 2 }
              },
              
              {
                  name: 'InvokeFlow',
                  props: { flowKey: 'xxxhhh' }
              },
          ]
      }
  ]
}


const transformer = compiler(node)

const ctx = {
    record: { id: '123', name: 'iphone' },
    route: { module: 'orders', action: 'show' },
}

console.log('1...')
await transformer(ctx)

console.log('2...')
await transformer(ctx)

compile {
  type: 'Flow',
  name: 'Pipeline',
  children: [
    { type: 'Flow', name: 'Validate', children: [Array] },
    { type: 'Flow', name: 'Compute', children: [Array] }
  ]
}
compile {
  type: 'Flow',
  name: 'Validate',
  children: [
    { name: 'ValidateData', props: [Object] },
    { name: 'ValidateService', props: [Object] }
  ]
}
compile { name: 'ValidateData', props: { name: 'iphone' } }
compile { name: 'ValidateService', props: { id: '123' } }
compile {
  type: 'Flow',
  name: 'Compute',
  children: [
    { name: 'InvokeService', props: [Object] },
    { name: 'InvokeService', props: [Object] },
    { name: 'InvokeFlow', props: [Object] }
  ]
}
compile {
  name: 'InvokeService',
  props: { serviceKey: 'loadProduct', param: 1 }
}
compile {
  name: 'InvokeService',
  props: { serviceKey: 'loadProduct', param: 2 }
}
compile { name: 'InvokeFlow', props: { flowKey: 'xxxhhh' } }
1...
invoke service, props: { serviceKey: 'loadProduct', param: 1 }, ctx: {
  record: { id: '123', n

## 待解决？

In [66]:
const { pipe, map, filter, tap, take, pick } = require('ramda')

In [45]:
//cats

In [98]:
await pipe(
    map(async x => ({ ...x, mate: faker.animal.bird() })),
    //tap(console.log),
    filter(x => x.age > 10),
) (cats)

[]


In [99]:
const Defers = list => ({
  filter: f => Defers(list.filter(x => x.then(f))),
  then: f => Promise.all(list).then(f),
})

In [124]:
compose(
    take(5),
    map(pick(['id', 'name'])),
    filter(x => x.age > 50),
) (cats)

// cats.length

[
  { id: '3e0e7325-94c1-4d9b-ab33-1b379d8f53be', name: 'Sokoke' },
  { id: '0eb9899f-d3ea-4a52-a2ce-33dd1c5df34e', name: 'Siberian' },
  { id: '579418f0-a0c4-4b11-9f63-0379a20b88a3', name: 'Balinese' },
  { id: 'd9a2c787-2adf-4439-95ac-df61e0e07de9', name: 'Birman' },
  { id: 'e1cd0f84-ed00-4693-9fe9-58e5ed49b399', name: 'Sokoke' }
]


In [120]:
const { compose, transduce, append, flip } = require('ramda')

In [125]:
const tr = compose(
    filter(x => {
        console.log(x)
        return x.age > 50
    }),
    map(pick(['id', 'name'])),
    take(5)
)

transduce(tr, flip(append), [], cats)

{
  id: '3e0e7325-94c1-4d9b-ab33-1b379d8f53be',
  name: 'Sokoke',
  age: 83,
  dark: false
}
{
  id: '2e5c6e40-68e8-4385-a046-a911280c055b',
  name: 'Maine Coon',
  age: 14,
  dark: true
}
{
  id: '0eb9899f-d3ea-4a52-a2ce-33dd1c5df34e',
  name: 'Siberian',
  age: 92,
  dark: true
}
{
  id: 'e89d8e37-3be7-47c7-8f41-94478f6c51da',
  name: 'Chausie',
  age: 15,
  dark: false
}
{
  id: 'b656012b-1b85-4c67-95db-f56e7ec2f045',
  name: 'Serengeti',
  age: 23,
  dark: false
}
{
  id: 'c53dd690-2943-4641-96cd-72d3448b0091',
  name: 'Tonkinese',
  age: 50,
  dark: false
}
{
  id: '579418f0-a0c4-4b11-9f63-0379a20b88a3',
  name: 'Balinese',
  age: 60,
  dark: true
}
{
  id: '710fad70-0c23-4326-8e25-626c0466a84b',
  name: 'Toyger',
  age: 12,
  dark: true
}
{
  id: 'b237e14b-5365-4ab8-854c-047ddb3c835b',
  name: 'Ragdoll',
  age: 30,
  dark: false
}
{
  id: '03a63c6a-f40d-4056-b40f-c85f36d9ddc8',
  name: 'Peterbald',
  age: 44,
  dark: false
}
{
  id: 'd9a2c787-2adf-4439-95ac-df61e0e07de9',
  name:

## typespec

In [272]:
// (a, b) -> a

type Reducer = <A, B>(a: A, b: B) => A

In [274]:
// a -> b
type Transformer = <A, B>(a: A) => B

In [314]:
const add = (a, b, c) => a + b + c
//add: a -> b -> c -> d

// reduce
// ((a, b) -> a) -> a -> [b] -> a